## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [151]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [152]:
df = pd.read_csv('CustomerReviews.csv')
df = df.drop(['Restaurant_ID','User'],axis=1)

In [153]:
df.tail()

,Review ID,Restaurant,Headline,Review,Rating
16,17,ข้าน้อยขอชาบู,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0
17,18,ข้าน้อยขอชาบู,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0
18,19,ข้าน้อยขอชาบู,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0
19,20,ข้าน้อยขอชาบู,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0
20,21,ข้าน้อยขอชาบู,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN


In [154]:
df['Restaurant'].unique()

array(['Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ',
       'Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G', 'ข้าน้อยขอชาบู'],
      dtype=object)

In [155]:
df = df[df['Restaurant']=='Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ']

## **Tokenize Words**

In [156]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ',';','+','-',':']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [157]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))
df['Headline_tokenized'] = df['Headline'].apply(lambda x: tokenize_with_space(x))

In [158]:
df.tail()

,Review ID,Restaurant,Headline,Review,Rating,Review_tokenized,Headline_tokenized
3,4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,"นึกถึง,ชาบู,เนื้อ,ราคา,โหด,เกินไป,นึกถึง,โม,โม...","เนื้อ,ดี,ไอติม,ดี,คุ้มค่า,ดี"
4,5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,"เดิน,หน้า,ได้กลิ่น,ชาบู,หอ,มมาก,โปร,บัตรเครดิต...","อาหาร,คุณภาพ,บริการ,ดี"
5,6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,กินให้อิ่ม ที่ momo,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,4.0,"บุ,ฟเฟ่,ชาบู,แนว,ญี่ปุ่น,สาย,เนื้อหมู,เนื้อวัว...","กิน,อิ่ม,momo"
6,7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ชาบู - สุกกี้สไลต์ญี่ปุ่นที่แท้ทรู,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,4.0,"Number,20,โม,โม,–,พารา,ได,ส์,สาขา,เดอะ,มอ,ล,กะ...","ชาบู,สุก,กี้,สไลต์,ญี่ปุ่น,ทรู"
7,8,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์ ที่ราคาเหมาะสมกับคุณภาพ ถือว่...,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,4.0,"ชาบู,เฟรนไชส์,รสชาติ,ดีมาก,คุ้มค่า,เหมาะสม,ราค...","ชาบู,เฟรนไชส์,ราคา,เหมาะสม,คุณภาพ,คุ้ม,น้ำ,ซุป..."


## **Create Dictionary**

In [159]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [160]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '', '++', '2', '555', '600', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่างกัน',

In [161]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [162]:
num_topics = 2
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 121 ms


In [163]:
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.063146  0.0       1        1  69.800346
1     -0.063146  0.0       2        1  30.199654, topic_info=         Term      Freq     Total Category  logprob  loglift
74       สาขา  3.000000  3.000000  Default  30.0000  30.0000
64     นึกถึง  2.000000  2.000000  Default  29.0000  29.0000
48          ​  9.000000  9.000000  Default  28.0000  28.0000
104       ชอบ  4.000000  4.000000  Default  27.0000  27.0000
178       โปร  1.000000  1.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
145  เนื้อวัว  0.793608  3.319640   Topic2  -5.2484  -0.2337
47      ไอติม  0.793540  3.319665   Topic2  -5.2485  -0.2338
3         กิน  0.793521  6.207087   Topic2  -5.2485  -0.8596
26       ราคา  0.793499  4.041534   Topic2  -5.2486  -0.4306
129       ลอง  0.793436  2.597848   Topic2  -5.2487   0.0113

[129 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
86        1  1.008844         
212       1  0.966669        /
88        1  0.779506        2
88        2  0.259835        2
169       2  0.866359        3
...     ...       ...      ...
180       2  0.866357       ไว
47        1  0.903706    ไอติม
47        2  0.301235    ไอติม
181       2  0.866358  ไอศกรีม
48        1  0.969064        ​

[107 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [164]:
model.show_topic(0)

[('น้ำ', 0.032053813),
 ('\u200b', 0.02585131),
 ('รสชาติ', 0.019646928),
 ('ซุป', 0.017577717),
 ('ชาบู', 0.015514107),
 ('เนื้อ', 0.015512368),
 ('กิน', 0.015511572),
 ('อร่อย', 0.015511469),
 ('เลือก', 0.0134423105),
 ('น้ำจิ้ม', 0.013442058)]

In [165]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [166]:
df[['Restaurant', 'Review', 'topics','Rating','score']]

,Restaurant,Review,topics,Rating,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,0,5.0,0.999633
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,1,5.0,0.999405
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,0,4.0,0.999823
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,1,4.0,0.999153
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,1,5.0,0.999301
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,1,4.0,0.999098
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,0,4.0,0.999583
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,0,4.0,0.999471
